<a href="https://colab.research.google.com/github/shichaog/GPT/blob/main/%E4%BD%BF%E7%94%A8_Sentencepiece%E6%89%A9%E5%85%85LLama_2%E4%B8%AD%E6%96%87%E8%AF%8D%E6%B1%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 首先使用Sentencepiece训练一个词汇表

This is a copy implementation of Sentencepiece example from google github. But with Chinese examples.


# mount Google Drive for reading《遮天》.txt

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/Colab Notebooks/
!ls zhe*

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks
zhetian.txt


# Install sentencepiece



In [2]:
import os

os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
!pip install sentencepiece
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.8 MB/s eta 0:00:00


In [3]:
from transformers import LlamaTokenizer

model_id="meta-llama/Llama-2-7b-hf"
model_id="NousResearch/Llama-2-7b-hf"
llama_tokenizer = LlamaTokenizer.from_pretrained(model_id)

# Basic end-to-end exmaple
Chinese don't use space to seprate words. The next code segments is using **unigram** method.When --model_type=unigram (default) is used, we can perform sampling and n-best segmentation for data augmentation. See subword regularization paper [kudo18] for more detail.

In [4]:
import sentencepiece as spm

# train sentencepiece model from `zhetian.txt` and makes `m.model` and `m.vocab`
# `m.vocab` is just a reference. not used in the segmentation.
spm.SentencePieceTrainer.train('--input=zhetian.txt --model_prefix=m --vocab_size=3439')

# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load('m.model')

# encode: text => id
print(sp.encode_as_pieces('叶凡经历九龙抬棺'))
print(sp.encode_as_ids('叶凡经历九龙抬棺'))

# decode: id => text
print(sp.decode_pieces(['▁', '叶', '凡', '经', '历', '九', '龙', '抬', '棺']))
print(sp.decode_ids([388, 359, 295, 606, 117]))

['▁', '叶', '凡', '经', '历', '九', '龙', '抬', '棺']
[6, 388, 359, 295, 606, 117, 101, 967, 383]
叶凡经历九龙抬棺
叶凡经历九


In [5]:
# returns vocab size
print(sp.get_piece_size())


# returns 0 for unknown tokens (we can change the id for UNK)
print(sp.piece_to_id('__MUST_BE_UNKNOWN__'))

# <unk>, <s>, </s> are defined by default. Their ids are (0, 1, 2)
# <s> and </s> are defined as 'control' symbol.
for id in range(3):
  print(sp.id_to_piece(id), sp.is_control(id))

3439
0
<unk> False
<s> True
</s> True


In [6]:
#list model output model
!ls m*
#check 20 line vocab
!head -n20 m.vocab

m_bpe.model  m_bpe.vocab  m.model  m.vocab

merged_tokenizer_hf:
special_tokens_map.json  tokenizer_config.json	tokenizer.model

merged_tokenizer_sp:
chinese_llama.model
<unk>	0
<s>	0
</s>	0
,	-2.2315
的	-3.60554
。	-3.71836
▁	-4.12704
了	-4.14309
”	-4.77899
在	-4.81982
一	-4.83639
不	-4.86514
大	-4.97118
有	-5.03706
他	-5.04904
是	-5.05205
人	-5.07435
道	-5.16509
天	-5.32164
都	-5.34114


其上是使用unigram 方式得到的vocab，其中第一个列是独立的单词（对应于Sentencepiece中的piece），第二列是这个词出现的对数概率值，对应于paper[https://aclanthology.org/P18-1007.pdf]中的公式6.

In [7]:
with open("zhetian.txt", "r", encoding="utf-8") as fp:
    data = fp.read().strip().split("\n")
sentences = []

for d in data:
    d = d.strip()
    if "===" in d or len(d) == 0 or d == "《遮天》来自:":
        continue
    sentences.append(d)

with open("corpus.txt", "w", encoding="utf-8") as fp:
    fp.write("\n".join(sentences))



In [8]:
spm.SentencePieceTrainer.train('--input=corpus.txt --model_prefix=m_bpe --vocab_size=3433 --model_type=bpe')

!head -n20 m_bpe.vocab

<unk>	0
<s>	0
</s>	0
,	-0
的	-1
▁	-2
。	-3
一	-4
了	-5
是	-6
不	-7
人	-8
有	-9
这	-10
“	-11
”	-12
他	-13
大	-14
在	-15
道	-16


In [9]:
from sentencepiece import sentencepiece_model_pb2 as sp_pb2_model

In [10]:
chinese_sp_model = spm.SentencePieceProcessor()
chinese_sp_model.Load('m.model')
chinese_spm = sp_pb2_model.ModelProto()
chinese_spm.ParseFromString(chinese_sp_model.serialized_model_proto())

278841

In [11]:
from huggingface_hub import login
login()

接下来导入LLama词汇表

In [12]:
import os
from transformers import LlamaTokenizer


In [13]:
from transformers import LlamaForCausalLM, LlamaTokenizer

import os

os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

model_id="meta-llama/Llama-2-7b-hf"
model_id="NousResearch/Llama-2-7b-hf"
tokenizer = LlamaTokenizer.from_pretrained(model_id)

In [14]:
llama_spm = sp_pb2_model.ModelProto()
llama_spm.ParseFromString(llama_tokenizer.sp_model.serialized_model_proto())

499723

In [15]:
# print number of tokens
print(len(llama_tokenizer),len(chinese_sp_model))
print(llama_tokenizer.all_special_tokens)
print(llama_tokenizer.all_special_ids)
print(llama_tokenizer.special_tokens_map)

32000 3439
['<s>', '</s>', '<unk>', '<unk>']
[1, 2, 0, 0]
{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}


In [16]:
## Add Chinese tokens to LLaMA tokenizer
llama_spm_tokens_set=set(p.piece for p in llama_spm.pieces)
print(len(llama_spm_tokens_set))
print(f"Before:{len(llama_spm_tokens_set)}")
for p in chinese_spm.pieces:
    piece = p.piece
    if piece not in llama_spm_tokens_set:
        new_p = sp_pb2_model.ModelProto().SentencePiece()
        new_p.piece = piece
        new_p.score = 0
        llama_spm.pieces.append(new_p)
print(f"New model pieces: {len(llama_spm.pieces)}")

32000
Before:32000
New model pieces: 34816


In [17]:
## Save
output_sp_dir = 'merged_tokenizer_sp'
output_hf_dir = 'merged_tokenizer_hf' # the path to save Chinese-LLaMA tokenizer
os.makedirs(output_sp_dir,exist_ok=True)
with open(output_sp_dir+'/chinese_llama.model', 'wb') as f:
    f.write(llama_spm.SerializeToString())
tokenizer = LlamaTokenizer(vocab_file=output_sp_dir+'/chinese_llama.model')

tokenizer.save_pretrained(output_hf_dir)
print(f"Chinese-LLaMA tokenizer has been saved to {output_hf_dir}")


# Test
llama_tokenizer = LlamaTokenizer.from_pretrained(model_id)
chinese_llama_tokenizer = LlamaTokenizer.from_pretrained(output_hf_dir)
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)
print(tokenizer.special_tokens_map)
text='''叶凡独自一人来到山前。
The primary use of LLaMA is research on large language models, including'''
print("Test text:\n",text)
print(f"Tokenized by LLaMA tokenizer:{llama_tokenizer.tokenize(text)}")
print(f"Tokenized by Chinese-LLaMA tokenizer:{chinese_llama_tokenizer.tokenize(text)}")

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Chinese-LLaMA tokenizer has been saved to merged_tokenizer_hf
['<s>', '</s>', '<unk>']
[1, 2, 0]
{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}
Test text:
 叶凡独自一人来到山前。
The primary use of LLaMA is research on large language models, including
Tokenized by LLaMA tokenizer:['▁', '<0xE5>', '<0x8F>', '<0xB6>', '<0xE5>', '<0x87>', '<0xA1>', '<0xE7>', '<0x8B>', '<0xAC>', '自', '一', '人', '来', '到', '山', '前', '。', '<0x0A>', 'The', '▁primary', '▁use', '▁of', '▁L', 'La', 'MA', '▁is', '▁research', '▁on', '▁large', '▁language', '▁models', ',', '▁including']
Tokenized by Chinese-LLaMA tokenizer:['叶', '凡', '独', '自', '一', '人', '来', '到', '山', '前', '。', '<0x0A>', 'The', '▁primary', '▁use', '▁of', '▁L', 'La', 'MA', '▁is', '▁research', '▁on', '▁large', '▁language', '▁models', ',', '▁including']
